In [ ]:
import pandas as pd
import driveami
import datetime
from driveami import keys as amikeys

In [ ]:
def ami_utc_to_datetimes(utc_str):
    date_format = '%Y-%m-%d %H:%M:%S'
    return datetime.datetime.strptime(utc_str, date_format)

In [ ]:
with open('./all_ami_rawfiles_metadata.json') as f:
    meta, _ = driveami.load_listing(f)
with open('./all_ami_rawfiles_by_pointing.json') as f:
    groupings, _ = driveami.load_listing(f)

In [ ]:
len(meta)

In [ ]:
len(groupings)

In [ ]:
#Drop rawtext
for k in meta.keys():
    meta[k].pop(amikeys.raw_obs_text, None)

# set groups
for group_id, group_dict in groupings.iteritems():
    for filename in group_dict[amikeys.files]:
        meta[filename][amikeys.group_name]= group_id

In [ ]:
df = pd.DataFrame.from_records(meta).T
df = df.drop([amikeys.comment,amikeys.pointing_hms_dms,amikeys.field],1)
df.head()

In [ ]:
df['utc_start'] = df.time_utc.apply(lambda x: ami_utc_to_datetimes(x[0]))
df['utc_end'] = df.time_utc.apply(lambda x: ami_utc_to_datetimes(x[1]))
df['mjd_start'] = df.time_mjd.apply(lambda x: x[0])
df['mjd_end'] = df.time_mjd.apply(lambda x: x[1])
df['ra'] = df.pointing_degrees.apply(lambda x: x[0])
df['dec'] = df.pointing_degrees.apply(lambda x: x[1])
df = df.drop([amikeys.time_ut, amikeys.time_mjd, amikeys.pointing_degrees], 1)


In [ ]:
now = datetime.datetime.now()
recent_obs = df[(now - df.utc_start) < datetime.timedelta(days=19)]
r = recent_obs[recent_obs.group_name.str.startswith('SWIFT')]
r

In [ ]:
r.T.to_dict()